In [ ]:
import project_path
import cogent3
import plotly.express as px

color_discrete_sequence = px.colors.sequential.Rainbow

FIG_DIMS = dict(width=800, height=600)

write_pdf = project_path.pdf_writer()


def load_table(fname, header):
    from cogent3.util.table import cast_str_to_array

    inpath = project_path.PAPER_DIR / "nbks" / "ctree" / "out" / fname
    lines = [l.split("\t") for l in inpath.read_text().splitlines()]
    data = dict(zip(header, zip(*lines)))
    for name, vals in data.items():
        data[name] = cast_str_to_array(vals)
    return cogent3.make_table(header=header, data=data)


def get_plot_for(*, x_axis, limit_to, colour, legend_title, log_x):
    table = load_table(
        "results_with_ls.tsv", ["k", "ss", "cpus", "time", "lnL", "tree"]
    )
    table = table.get_columns([c for c in table.header if c != "tree"])
    table = table.filtered(lambda x: int(x) in limit_to, columns=colour)
    iqtree = load_table("iqtree_with_ls.tsv", ["time", "lnL", "tree"])
    iqtree_lnL = iqtree.columns["lnL"][0]
    fig = px.line(
        table,
        x=x_axis,
        y="lnL",
        color=colour,
        labels={"ss": "Sketch Size", "k": "<i>k</i>"},
        color_discrete_sequence=color_discrete_sequence,
        log_x=log_x,
    )

    fig.add_hline(
        y=iqtree_lnL,
        line_dash="dash",
        line_color="black",
        annotation_text="IQ-TREE",
        annotation_position="bottom left",
    )

    fig.update_layout(legend_title_text=legend_title)

    diff = 0.005
    max_val = (1 - diff) * iqtree_lnL
    min_val = (1 + diff) * table.columns["lnL"].min()
    fig.update_yaxes(autorange=False, range=[min_val, max_val])
    fig.update_layout(**FIG_DIMS)
    return fig


def make_likelihood_vs_k():
    return get_plot_for(
        x_axis="ss",
        colour="k",
        legend_title="<i>k</i>",
        limit_to=[8, 10, 12, 14, 16],
        log_x=True,
    )


def make_likelihood_vs_ss():
    return get_plot_for(
        x_axis="k",
        colour="ss",
        legend_title="<b>Sketch Size</b>",
        limit_to=[100, 1000, 10_000, 25_000, 100000],
        log_x=False,
    )

In [ ]:
fig = make_likelihood_vs_k()
fig.show()
# write_pdf(fig, project_path.FIG_DIR / "likelihood_vs_ss_for_k.pdf")

In [ ]:
fig = make_likelihood_vs_ss()
fig.show()
# write_pdf(fig, project_path.FIG_DIR / "likelihood_vs_k_for_ss.pdf")